In [2]:
import sscv 
import mmcv 
import matplotlib.pyplot as plt 
import os.path as osp
import numpy as np

In [3]:
from glob import glob 
from PIL import Image
from mmseg.apis import init_segmentor, inference_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette

In [4]:
config_file = '/home/soojin/UOS-SSaS Dropbox/05. Data/03. Checkpoints/2020.09.02_deeplabv3plus_r101-d8_769x769_40k_concrete_damage_cs/deeplabv3plus_r101-d8_769x769_40k_concrete_damage_cs.py'
checkpoint_file = '/home/soojin/UOS-SSaS Dropbox/05. Data/03. Checkpoints/2020.09.02_deeplabv3plus_r101-d8_769x769_40k_concrete_damage_cs/iter_40000.pth'

In [5]:
# build the model from a config file and a checkpoint file
model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

/home/soojin/anaconda3/envs/auto-label/lib/python3.8/site-packages/mmseg/models/builder.py:59: UserWarning: train_cfg and test_cfg is deprecated, please specify them in model
  warnings.warn(


Use load_from_local loader


In [6]:
auto_label_dir = "/home/soojin/UOS-SSaS Dropbox/05. Data/06. Auto Learning/"

dataset_name = 'General Crack'
raw_img_dir_name = '01. Raw Image'
rsz_img_dir_name = '02. Resized Image'
detection_result_dir_name = '01. Detection Result'


dataset_dir = osp.join(auto_label_dir, dataset_name)
raw_img_dir = osp.join(dataset_dir, raw_img_dir_name)
rsz_img_dir = osp.join(dataset_dir, rsz_img_dir_name)
detection_result_dir = osp.join(rsz_img_dir, detection_result_dir_name)

In [10]:
rsz_img_list

[]

In [9]:
rsz_img_list = [osp.basename(x) for x in glob(osp.join(rsz_img_dir, '*.jpg'))]

for idx, img_filename in enumerate(rsz_img_list): 
    img_filepath = osp.join(rsz_img_dir, img_filename)
    img_basename = osp.basename(img_filename).split('.')[0]
    img = sscv.imread(img_filepath)
    
    result = inference_segmentor(model, img_filepath)
    
    detection_result_with_transparent_background = np.ones(img.shape, dtype = np.uint8)*255 # create white image 
    
    crack_area = result[0] == 1
    detection_result_with_transparent_background[crack_area, 0] =  255
    detection_result_with_transparent_background[crack_area, 1] =  0
    detection_result_with_transparent_background[crack_area, 2] =  0
    
    detection_result_with_transparent_background = Image.fromarray(detection_result_with_transparent_background) 
    detection_result_with_transparent_background = detection_result_with_transparent_background.convert("RGBA")
    
    img_datas = detection_result_with_transparent_background.getdata()
    
    newData = []
    
    for item in img_datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)
  
    detection_result_with_transparent_background.putdata(newData)
    detection_result_with_transparent_background.save(osp.join(detection_result_dir, img_basename + '_crack.png'), "PNG")